In [ ]:
#1 зробити хом пейдж (текст, картинки)
#2 зробити сторінку з новинами
#3 сторінку з рекомендаціями
#4 додати більше можливостей для карти і графіка

In [ ]:
# сторінка з новинами

In [ ]:
# сторінка з рекомендаціями

In [ ]:
import pandas as pd
import folium
import streamlit as st
from matplotlib import pyplot as plt
from streamlit_folium import st_folium, folium_static


# текст на боковій панелі
sidebar_header = """
    <div style='text-align: center;'>
        <h1>Забруднення повітря</h1>
        <p>Забруднення повітря є серйозною проблемою в сучасному світі, що може впливати на здоров'я людей та навколишнє середовище. На цьому сайті ви зможете обрати конкретну країну, рік, забруднювач та подивитися на карті та графіку як багато його в певній місцевості
        </div>
"""

st.sidebar.markdown(sidebar_header, unsafe_allow_html=True)


# загрузка даних
@st.cache_data
def load_data():
    df_air = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/openaq.xlsx")
    df_geo = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/worldcities1.xlsx")
    df = pd.merge(df_air, df_geo, left_on=['Country Code', 'City'], right_on=[
                  'iso2', 'City'], how='left')
    df = df.dropna(subset=['Coordinates'])
    df[['Lat', 'Lon']] = df['Coordinates'].str.split(
        ',', expand=True).astype(float)
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], utc=True)
    return df

df = load_data()


# бокова панель
with st.sidebar:
    country = st.selectbox("Оберіть країну", df["Country Label"].unique())
    year_options = df[df["Country Label"] ==
                      country]["Last Updated"].dt.year.unique()
    if len(year_options) > 0:
        year = st.selectbox("Оберіть рік", year_options)
    else:
        st.write("No data available for selected country.")
        year = None
    pollutant_options = df['Pollutant'].unique()
    pollutant = st.multiselect("Оберіть забруднювач", pollutant_options)


# фільтруємо обрані дані на боковій панелі
if year is not None:
    inCountry = df["Country Label"] == country
    withCords = df['Lon'].notna() & df['Lat'].notna()
    selectedPolutants = df['Pollutant'].isin(pollutant)
    selectedYear = df['Last Updated'].dt.year == year
    filtered_df = df[inCountry & withCords & selectedPolutants & selectedYear]

    # карта
    st.markdown("## Карта {} в {}".format(", ".join(pollutant), country))

    if len(filtered_df) > 0:
        map_air = folium.Map(
            location=[filtered_df['Lat'].iloc[0], filtered_df['Lon'].iloc[0]], zoom_start=10)
        map_air.fit_bounds([[filtered_df['Lat'].min(), filtered_df['Lon'].min()], [
                           filtered_df['Lat'].max(), filtered_df['Lon'].max()]])

       # позначки на карті
        for index, row in filtered_df.iterrows():
            if row['Pollutant'] in pollutant:
                folium.Marker(
                    [row['Lat'], row['Lon']],
                    color='red',
                    tooltip="{:.2f}{} {}".format(
                        row['Value'], row['Unit'], row['Pollutant'])
                ).add_to(map_air)


        st_data = st_folium(map_air, width=700)

    else:
        st.write("No data available for selected inputs.")


# графік
if year is not None:

    filtered_df[['City','Pollutant','Value']].groupby(['City','Pollutant']).max().unstack().plot.bar(y='Value')

    plt.xlabel('City')
    plt.ylabel('Pollution level')
    plt.xticks(rotation=90)
    st.pyplot(plt)

In [ ]:
#типо кнопка для загрузки пдф файла
    map_html = st_data._repr_html_()
    map_img = imgkit.from_string(map_html, False, options={"zoom": 1.5})
    chart_img = imgkit.from_string(st.pyplot(plt).get_figure(), False)


    def create_download_button(html):
        def save_pdf(html, filename):
            pdfkit.from_string(html, filename)

        def download_pdf():
            save_pdf(html, "maps_and_charts.pdf")
            with open("maps_and_charts.pdf", "rb") as f:
                pdf_bytes = f.read()
            st.download_button("Download PDF file", pdf_bytes, mime="application/pdf")

        download_pdf()


    # ...

    map_html = st_data._repr_html_()
    map_img = imgkit.from_string(map_html, False, options={"zoom": 1.5})

    chart_html = st.pyplot(plt).to_html()
    chart_img = imgkit.from_string(chart_html, False)

    html = f"""
    <html>
        <head>
            <meta charset="UTF-8">
            <title>Maps and Charts</title>
        </head>
        <body>
            <h1>Maps of {", ".join(pollutant)} in {country}</h1>
            <img src="data:image/png;base64,{map_img}" />
            <h1>Pollution levels in {country}</h1>
            <img src="data:image/png;base64,{chart_img}" />
        </body>
    </html>
    """

    create_download_button(html)

In [ ]:
# сторінка з картой і графіком
# загрузка даних
@st.cache_data
def load_data():
    df_air = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/openaq.xlsx")
    df_geo = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/worldcities1.xlsx")
    df = pd.merge(df_air, df_geo, left_on=['Country Code', 'City'], right_on=[
                  'iso2', 'City'], how='left')
    df = df.dropna(subset=['Coordinates'])
    df[['Lat', 'Lon']] = df['Coordinates'].str.split(
        ',', expand=True).astype(float)
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], utc=True)
    return df

df = load_data()


# бокова панель
with st.sidebar:
    country = st.selectbox("Оберіть країну", df["Country Label"].unique())
    year_options = df[df["Country Label"] ==
                      country]["Last Updated"].dt.year.unique()
    if len(year_options) > 0:
        year = st.selectbox("Оберіть рік", year_options)
    else:
        st.write("No data available for selected country.")
        year = None
    pollutant_options = df['Pollutant'].unique()
    pollutant = st.multiselect("Оберіть забруднювач", pollutant_options)


# фільтруємо обрані дані на боковій панелі
if year is not None:
    inCountry = df["Country Label"] == country
    withCords = df['Lon'].notna() & df['Lat'].notna()
    selectedPolutants = df['Pollutant'].isin(pollutant)
    selectedYear = df['Last Updated'].dt.year == year
    filtered_df = df[inCountry & withCords & selectedPolutants & selectedYear]

    # карта
    st.markdown("## Карта {} в {}".format(", ".join(pollutant), country))

    if len(filtered_df) > 0:
        map_air = folium.Map(
            location=[filtered_df['Lat'].iloc[0], filtered_df['Lon'].iloc[0]], zoom_start=10)
        map_air.fit_bounds([[filtered_df['Lat'].min(), filtered_df['Lon'].min()], [
                           filtered_df['Lat'].max(), filtered_df['Lon'].max()]])

       # позначки на карті
        for index, row in filtered_df.iterrows():
            if row['Pollutant'] in pollutant:
                folium.Marker(
                    [row['Lat'], row['Lon']],
                    color='red',
                    tooltip="{:.2f}{} {}".format(
                        row['Value'], row['Unit'], row['Pollutant'])
                ).add_to(map_air)


        st_data = st_folium(map_air, width=700)

    else:
        st.write("No data available for selected inputs.")


# графік
if year is not None:

    filtered_df[['City','Pollutant','Value']].groupby(['City','Pollutant']).max().unstack().plot.bar(y='Value')

    plt.xlabel('City')
    plt.ylabel('Pollution level')
    plt.xticks(rotation=90)
    st.pyplot(plt)


In [ ]:
country = st.selectbox("Оберіть країну", df["Country Label"].unique())
year_options = df[df["Country Label"] == country]["Last Updated"].dt.year.unique()
    if len(year_options) > 0:
            year = st.selectbox("Оберіть рік", year_options)
    else:
        st.write("No data available for selected country.")
        year = None
    pollutant_options = df['Pollutant'].unique()
    pollutant = st.multiselect("Оберіть забруднювач", pollutant_options)


    # фільтруємо обрані дані на боковій панелі
    if year is not None:
        inCountry = df["Country Label"] == country
        withCords = df['Lon'].notna() & df['Lat'].notna()
        selectedPolutants = df['Pollutant'].isin(pollutant)
        selectedYear = df['Last Updated'].dt.year == year
        filtered_df = df[inCountry & withCords & selectedPolutants & selectedYear]

        # кнопка для отображения карты и графика
        button = st.button("Показати карту та графік")

        if button:

            st.markdown("## Карта {} в {}".format(", ".join(pollutant), country))

            if len(filtered_df) > 0:
                map_air = folium.Map(location=[filtered_df['Lat'].iloc[0], filtered_df['Lon'].iloc[0]], zoom_start=10)
                map_air.fit_bounds([[filtered_df['Lat'].min(), filtered_df['Lon'].min()], [filtered_df['Lat'].max(), filtered_df['Lon'].max()]])

                # позначки на карті
                for index, row in filtered_df.iterrows():
                    if row['Pollutant'] in pollutant:
                        folium.Marker(
                            [row['Lat'], row['Lon']],
                            color='red',
                            tooltip="{:.2f}{} {}".format(
                                row['Value'], row['Unit'], row['Pollutant'])
                        ).add_to(map_air)

                st_data = st_folium(map_air, width=700)

            else:
                st.write("No data available for selected inputs.")

            # графік
            filtered_df[['City', 'Pollutant', 'Value']].groupby(['City', 'Pollutant']).max().unstack().plot.bar(
                y='Value')

            plt.xlabel('Місто')
            plt.ylabel('Рівень забрудності')
            plt.xticks(rotation=90)
            st.pyplot(plt)

In [ ]:
# КАРТА ТА ГРАФІК
if select_page == 'Карта і графік':
    st.title('Карта і графік')

    # загрузка даних
    @st.cache_resource
    def load_data():
        df_air = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/mkalokhina/openaq.xlsx")
        df_geo = pd.read_excel("C:/Users/kalok/OneDrive/Робочий стіл/mkalokhina/worldcities1.xlsx")
        df = pd.merge(df_air, df_geo, left_on=['Country Code', 'City'], right_on=['iso2', 'City'], how='left')
        df = df.dropna(subset=['Coordinates'])
        df[['Lat', 'Lon']] = df['Coordinates'].str.split(',', expand=True).astype(float)
        df['Last Updated'] = pd.to_datetime(df['Last Updated'], utc=True)
        return df

    df = load_data()


    # вибір опцій
    countries = st.multiselect("Оберіть країни", df["Country Label"].unique())
    year_options = df[df["Country Label"].isin(countries)]["Last Updated"].dt.year.unique()
    if len(year_options) > 0:
        year = st.selectbox("Оберіть рік", year_options)
    else:
        st.write("No data available for selected countries.")
        year = None
    pollutant_options = df['Pollutant'].unique()
    pollutant = st.multiselect("Оберіть забруднювач", pollutant_options)


    # фільтруємо обрані дані на боковій панелі
    if year is not None:
        inCountries = df["Country Label"].isin(countries)
        withCords = df['Lon'].notna() & df['Lat'].notna()
        selectedPolutants = df['Pollutant'].isin(pollutant)
        selectedYear = df['Last Updated'].dt.year == year
        filtered_df = df[inCountries & withCords & selectedPolutants & selectedYear]

        # кнопка для отображения карты і графіка
        button = st.button("Показати карту та графік")

        if button:
            st.markdown("## Карта {} в {}".format(", ".join(pollutant), ", ".join(countries)))

            if len(filtered_df) > 0:
                map_air = folium.Map(location=[filtered_df['Lat'].iloc[0], filtered_df['Lon'].iloc[0]], zoom_start=3)

                for index, row in filtered_df.iterrows():
                    if row['Pollutant'] in pollutant:
                        folium.Marker(
                            [row['Lat'], row['Lon']],
                            color='red',
                            tooltip="{:.2f}{} {}, {}".format(
                                row['Value'], row['Unit'], row['Pollutant'], row['City'])
                        ).add_to(map_air)

                st_data = st_folium(map_air, width=700)

            else:
                st.write("No data available for selected inputs.")

            # графік
            fig, ax = plt.subplots()
            filtered_df[['City', 'Pollutant', 'Value']].groupby(['City', 'Pollutant']).max().unstack().plot.bar(
                y='Value', ax=ax)

            ax.set_xlabel('Місто')
            ax.set_ylabel('Рівень забрудності')
            ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
            st.pyplot(fig)